In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
from datetime import datetime
import argparse
import psutil
import numpy as np
import logging
import numpy as np
import tensorflow as tf
import time
import glob
from tensorflow.python.framework import ops
import math
from dotenv import find_dotenv, load_dotenv
import ftplib

from scipy.stats.stats import pearsonr

nb_dir = os.path.join(Path(os.getcwd()).parents[0], 'src', 'data')
if nb_dir not in sys.path:
    sys.path.insert(0, nb_dir)
print(sys.path)
import features_selection as fs
import make_dataset as md
import build_data as bd
import get_raw_data as grd

models_dir = os.path.join(Path(os.getcwd()).parents[0], 'src', 'models')
if models_dir not in sys.path:
    sys.path.insert(0, models_dir)
import nn_real as nn

load_dotenv(find_dotenv())

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
pd.set_option('io.hdf.default_format','table')

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
2018-11-29 18:36:43,634 - matplotlib - DEBUG - CACHEDIR=/home/ubuntu/.cache/matplotlib
2018-11-29 18:36:43,638 - matplotlib.font_manager - DEBUG - Using fontManager instance from /home/ubuntu/.cache/matplotlib/fontList.json
2018-11-29 18:36:43,718 - matplotlib.backends - DEBUG - backend module://ipykernel.p

['/home/ubuntu/MLMortgage/src/data', '', '/home/ubuntu/src/cntk/bindings/python', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python36.zip', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/lib-dynload', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions', '/home/ubuntu/.ipython']


2018-11-29 18:36:43,785 - matplotlib.backends - DEBUG - backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

RAW_DIR = os.path.join(Path(os.getcwd()).parents[0], 'data', 'raw') 
PRO_DIR = os.path.join(Path(os.getcwd()).parents[0], 'data', 'processed')
RANDOM_SEED = 123  # Set the seed to get reproducable results.
DT_FLOAT = np.float32
NP_FLOAT = np.dtype('float32')

train_dir = 'chuncks_random_c1millx2_train' # 'chuncks_random_1th_train'
valid_dir = 'chuncks_random_c1millx2_valid' # 'chuncks_random_1th_valid'
test_dir = ''
train_period=[121,323] #[121,279] #[121, 143] 
valid_period=[324,329] #[280,285] #[144, 147] 
test_period=[330,342] #[286,304] #[148, 155]
batch_size=50000


print(RAW_DIR, PRO_DIR)

/home/ubuntu/MLMortgage/data/raw /home/ubuntu/MLMortgage/data/processed


In [3]:
def get_files_dict(train_dir, valid_dir, test_dir):        
    ext = "*.h5"

    files_dict = {'train': glob.glob(os.path.join(PRO_DIR, train_dir, ext)), 
                  'valid': glob.glob(os.path.join(PRO_DIR, valid_dir, ext)), 
                  'test': glob.glob(os.path.join(PRO_DIR, test_dir, ext))}

    return files_dict

def architecture_settings(files_dict):
    architecture = {}    
    ok_inputs = True
    
    for key in files_dict.keys():
        total_records = 0
        for file in files_dict[key]:                                
            with pd.HDFStore(file) as dataset_file:
                if (ok_inputs): 
                    index_length = len(dataset_file.get_storer(key+'/features').attrs.data_columns)
                    architecture['n_input'] = dataset_file.get_storer(key+ '/features').ncols - index_length
                    architecture['n_classes'] = dataset_file.get_storer(key+'/labels').ncols - index_length
                    ok_inputs = False                
                total_records += dataset_file.get_storer(key + '/features').nrows
        architecture[key + '_num_examples'] = total_records                            
    
    architecture['total_num_examples'] = architecture['train_num_examples'] # 10000000
    return architecture

In [4]:
#To sum up the dataset per worker (assuming the same size of files per worker approximately):
files_dict = get_files_dict(train_dir, valid_dir, test_dir)
architecture = architecture_settings(files_dict)
print(architecture)

{'n_input': 258, 'n_classes': 7, 'train_num_examples': 26769588, 'valid_num_examples': 901868, 'test_num_examples': 0, 'total_num_examples': 26769588}


In [5]:
# Saving into another .h5 dataset:
def remove_label(tag, label_todel, file_dir, fname):    
    all_files = glob.glob(os.path.join(PRO_DIR, file_dir, "*.h5"))
    file_name = fname + tag + '_.h5'
    target_path = os.path.join(PRO_DIR, file_dir, file_name)          
    with pd.HDFStore(target_path, complib='lzo', complevel=9) as hdf_target:
        print('Target Path: ', target_path)   
        # idxs = np.resize(drop_cols.values(),np.count_nonzero(drop_cols.values()))                
        # print('columns dropped')
        acc_rows = 0        
        total_files = len(all_files)            
        lab_classes = ['0', '3', '6', '9', 'C', 'F', 'R']
        total_num_examples = 0        
        for i, file_path in zip(range(total_files), all_files):    
            with pd.HDFStore(file_path) as dataset_file:                                
                print(file_path, '...to load')
                #columns = dataset_file.get_storer(dtype+'/labels').attrs.non_index_axes[0][1]
                total_records_file=0
                for nclass in (set(lab_classes) - set(label_todel)):
                    print('class No.: ', nclass)                       
                    class_features = dataset_file.select(tag+'/features',  "index=='"+ str(nclass) + "'")                    
                    class_features = class_features.astype(DT_FLOAT)
                    class_labels = dataset_file.select(tag+'/labels',  "index=='"+ str(nclass) + "'")
                    class_labels.drop('DELINQUENCY_STATUS_NEXT_'+label_todel, axis=1, inplace=True)
                    class_labels = class_labels.astype(np.int8)
                    total_rows = len(class_labels.values)
                    hdf_target.put(tag + '/features', class_features, append=True) 
                    hdf_target.put(tag + '/labels', class_labels, append=True) 
                    hdf_target.flush()
                    print(file_path, ' class ', nclass,': saved in disk, nrows: ', total_rows, 'labcols: ', class_labels.shape[1])                                                            
                    total_num_examples += total_rows                    
                                    
        print('total_num_examples: ', total_num_examples)
        


In [6]:
label_todel = 'C'
remove_label('train', label_todel, train_dir, '_dataset_non_current_')

Target Path:  /home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_train/dataset_non_current_train_.h5
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_train/temporalloandynmodifmrstaticitur15mill-16mill-train_0.h5 ...to load
class No.:  6
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_train/temporalloandynmodifmrstaticitur15mill-16mill-train_0.h5  class  6 : saved in disk, nrows:  259623 labcols:  6
class No.:  9
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_train/temporalloandynmodifmrstaticitur15mill-16mill-train_0.h5  class  9 : saved in disk, nrows:  614496 labcols:  6
class No.:  R
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_train/temporalloandynmodifmrstaticitur15mill-16mill-train_0.h5  class  R : saved in disk, nrows:  27885 labcols:  6
class No.:  F
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_train/temporalloandynmodifmrstaticitur15mill-16mill-train_0.h5  class  F : saved in disk, nr

In [7]:
remove_label('valid', label_todel, valid_dir, '_dataset_non_current_')

Target Path:  /home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_valid/dataset_non_current_valid_.h5
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_valid/temporalloandynmodifmrstaticitur15mill-16mill-valid_0.h5 ...to load
class No.:  6
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_valid/temporalloandynmodifmrstaticitur15mill-16mill-valid_0.h5  class  6 : saved in disk, nrows:  6681 labcols:  6
class No.:  9
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_valid/temporalloandynmodifmrstaticitur15mill-16mill-valid_0.h5  class  9 : saved in disk, nrows:  15002 labcols:  6
class No.:  R
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_valid/temporalloandynmodifmrstaticitur15mill-16mill-valid_0.h5  class  R : saved in disk, nrows:  410 labcols:  6
class No.:  F
/home/ubuntu/MLMortgage/data/processed/chuncks_random_c1millx2_valid/temporalloandynmodifmrstaticitur15mill-16mill-valid_0.h5  class  F : saved in disk, nrows: 